In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig
from torch.nn.functional import softmax


/home/mchorse/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
dataset = pd.read_hdf("memorized-data/19m.hdf").sample(100)
dataset

,index,accuracy,tokens
556518,98940518,1.0,"[1394, 778, 4044, 247, 3491, 273, 253, 4637, 3..."
797185,140365185,1.0,"[431, 1019, 8402, 94, 187, 50262, 61, 2099, 92..."
700522,21292522,1.0,"[64, 11364, 1848, 8286, 64, 7310, 50261, 30, 4..."
149601,41333601,1.0,"[387, 6831, 2363, 273, 4398, 938, 15, 17, 1584..."
67235,80147235,1.0,"[4270, 327, 253, 2800, 7625, 11443, 8604, 60, ..."
...,...,...,...
1570694,45042694,1.0,"[568, 37, 26649, 337, 15, 25, 15, 883, 5647, 1..."
1258867,131674867,1.0,"[938, 3725, 47279, 8604, 60, 805, 431, 1019, 8..."
1850610,49898610,1.0,"[187, 1911, 634, 1211, 11743, 187, 187, 1911, ..."
170213,142026213,1.0,"[187, 249, 253, 9107, 1293, 12400, 13, 1690, 1..."


In [3]:
device = torch.device("cuda")
pythia_model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-70m")
pythia_model.to(device)
pythia_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

In [4]:
# def get_perplexity(dataset_index):
tokens = dataset.iloc[0]["tokens"]
text = pythia_tokenizer.decode(tokens)
model_inputs = pythia_tokenizer(text, return_tensors="pt").to(device)
target_ids = model_inputs.input_ids.clone().to(device)
# pythia_model.generate(**model_inputs, return_dict_in_generate=True, output_scores=True)
pythia_model(**model_inputs, labels=target_ids)


CausalLMOutputWithPast(loss=tensor(0.3538, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[94.1922, 56.2604, 98.2960,  ..., 56.2614, 56.2595, 56.2630],
         [96.4956, 54.3007, 97.5315,  ..., 54.3010, 54.2996, 54.3022],
         [94.7728, 55.3570, 93.9256,  ..., 55.3577, 55.3552, 55.3580],
         ...,
         [47.5782, 19.7386, 42.1811,  ..., 19.7412, 19.7374, 19.7403],
         [37.4160, 19.5540, 33.2688,  ..., 19.5539, 19.5515, 19.5540],
         [53.2555, 24.4876, 54.1368,  ..., 24.4899, 24.4872, 24.4893]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-8.7410e-02,  1.4655e+00,  1.4457e+00,  ..., -5.1421e+00,
            1.1504e+00,  7.0371e-01],
          [-9.6436e-02, -5.2639e-01,  8.9948e-01,  ..., -4.5567e+00,
            3.2489e-01,  1.7674e+00],
          [ 9.1574e-02, -4.2861e-01,  8.3932e-02,  ..., -3.6315e+00,
            1.0319e+00,  2.4560e+00],
          ...,
          [ 8.3537e-01, -5.1634e-02,  6.5463e-01,  .

In [8]:
for sequence_record in dataset[0:1].iterrows():
    pile_index = sequence_record[1]["index"]
    tokens = sequence_record[1]["tokens"]

    for token_index, true_token in enumerate(tokens):
        if token_index == 0:
            continue

        context = tokens[0:token_index]
        text = pythia_tokenizer.decode(context, return_tensors="pt")
        model_inputs = pythia_tokenizer(text, return_tensors="pt").to(device)

        true_context = tokens[0:token_index + 1]
        true_text = pythia_tokenizer.decode(true_context, return_tensors="pt")
        target_ids = pythia_tokenizer(true_text, return_tensors="pt").input_ids.to(device)
        
        target_ids[:, :-token_index] = -100

        losses = np.empty(len(tokens))

        with torch.no_grad():
            output = pythia_model(**model_inputs, labels=target_ids)
            loss = output.loss
        
        print(losses)
            


Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using sep_token, but it is not set yet.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1438151/1396924949.py:22 in <module>                                              │
│                                                                                                  │
│   19 │   │   losses = np.empty(len(tokens))                                                      │
│   20 │   │                                                                                       │
│   21 │   │   with torch.no_grad():                                                               │
│ ❱ 22 │   │   │   output = pythia_model(**model_inputs, labels=target_ids)                        │
│   23 │   │   │   loss = output.loss                                                              │
│   24 │   │                                                                                       │
│   25 │   │   print(losses)                                                                       │
│                                                                                                  │
│ /home/mchorse/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1130 in _call_impl   │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/mchorse/.local/lib/python3.9/site-packages/transformers/models/gpt_neox/modeling_gpt_neox. │
│ py:675 in forward                                                                                │
│                                                                                                  │
│   672 │   │   │   shift_logits = lm_logits[:, :-1, :].contiguous()                               │
│   673 │   │   │   labels = labels[:, 1:].contiguous()                                            │
│   674 │   │   │   loss_fct = CrossEntropyLoss()                                                  │
│ ❱ 675 │   │   │   lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view   │
│   676 │   │                                                                                      │
│   677 │   │   if not return_dict:                                                                │
│   678 │   │   │   output = (lm_logits,) + outputs[1:]                                            │
│                                                                                                  │
│ /home/mchorse/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1130 in _call_impl   │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks